## Parallel Tool Use with Groq API

To extend the capabilities of Large Language Models (LLMs) in AI-powered applications and systems, we can provide tools to allow them to interact with external resources (e.g. APIs, databases, web) by:
    <ul>
    <li>Providing tools (or predefined functions) to our LLM</li>
    <li>Defining how the tools we provide should be used to teach our LLM how to use them effectively (e.g. defining input and output formats) </li>
    <li>
    Letting the LLM autonomously decide whether or not the provided tools are needed for a user query by evaluating the user query, determining whether the tools can enhance its response, and utilizing the tools accordingly</li>
    </ul>


##### Tool use, or function calling, support is available for all text models and parallel tool use support is enabled for all Llama 3 and Llama 3.1 models. The Llama 3.1 models now support the native tool use format that was used in post-training, which results in much better quality, especially in multi-turn conversations and parallel tool calling.

In [1]:
import os
import json

from groq import Groq
from dotenv import load_dotenv

load_dotenv()
"Groq API key configured: " + os.environ["GROQ_API_KEY"][:10] + "..."

'Groq API key configured: gsk_Et8zyU...'